In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
 
docs=[]
docs_names=[]

path="/content/drive/MyDrive/FYP/Text_Files"
dir_list = os.listdir(path)
print(dir_list)

for f in dir_list:
  with open(path+"/"+f, "r", encoding='Windows-1252') as file:
    data = file.read()
  data = data.replace("\n"," ")
  docs.append(data)
  docs_names.append(f)
  #print(f)

In [ ]:
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(docs)

print(X.shape)

In [ ]:
df = pd.read_csv ('/content/drive/MyDrive/FYP/dataset.csv')

In [ ]:
df

In [ ]:
from scipy.sparse import csr_matrix

S = csr_matrix(X)

# reconstruct dense matrix
Z = S.todense()

In [ ]:
Z[99].shape

In [ ]:
from keras.models import load_model
model = load_model('/content/drive/MyDrive/FYP/saved_model/autoencoder.h5')

In [ ]:
import keras
from keras.layers import Dense,Conv1D,UpSampling1D,MaxPooling1D
from keras.models import sequential
from keras import *

In [ ]:
from tensorflow.keras.utils import plot_model

# define an encoder model (without the decoder)
encoder = Model(inputs=model.input, outputs=model.layers[-9].output)
#plot_model(encoder, 'encoder_no_compress.png', show_shapes=True)


In [ ]:
compress = []
for i in range(len(Z)) :
  a = (encoder.predict(Z[i]).flatten())
  compress.append(a)

In [ ]:
len(compress[0])

In [ ]:
print(np.array(compress[0]))

In [ ]:
doc1 = []

for i in range(len(df)) :
  a = docs_names.index(df.iloc[i,0])
  doc1.append(compress[a])

df['doc1'] = doc1

In [ ]:
doc2 = []

for i in range(len(df)) :
  a = docs_names.index(df.iloc[i,1])
  doc2.append(compress[a])

df['doc2'] = doc2

In [ ]:
df.head()

In [ ]:
doc1= df['doc1']

In [ ]:
doc2= df['doc2']

In [ ]:
label = df['Average']

In [ ]:
doc1= tf.convert_to_tensor(doc1)

In [ ]:
doc2= tf.convert_to_tensor(doc2)

In [ ]:
from keras.layers import Concatenate

input1 = Input(shape=(3930,))
input2 = Input(shape=(3930,))
input = Concatenate()([input1, input2])
x = Dense(256)(input)
x = Dense(128)(input)
x = Dense(128)(input)
x = Dense(1)(x)
model1 = Model(inputs=[input1, input2], outputs=x)
model1.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model1.compile(loss= 'mse', metrics=["mse"], optimizer=opt)
model1.fit([doc1 , doc2], label , epochs = 10, batch_size = 2)

In [ ]:
from tensorflow.keras.models import load_model

model1.save('/content/drive/MyDrive/FYP/saved_model/siamesemodel.h5')